In [1]:
using JOLI
using Waveform

In [ ]:
n = 50*[1;1;1];
d = 12.5*[1;1;1];
o = 0.0*[1;1;1];
v0 = 2000;
freq = 4.0;
λ = v0/freq;

t0 = 0.0;
f0 = 10.0;
unit = "m/s";
xsrc = [(n.*d)[1]/2];
ysrc = [(n.*d)[2]/2];
zsrc = [100.0];
xrec = linspace(0.0,(n.*d)[1]::Float64,n[1]);
yrec = linspace(0.0,(n.*d)[2]::Float64,n[2]);
zrec = [100.0];
freqs = [freq];
nsrc = length(xsrc)*length(ysrc)*length(zsrc);
nfreq = length(freqs)
model = Model{Int64,Float64}(n,d,o,t0,f0,unit,freqs,xsrc,ysrc,zsrc,xrec,yrec,zrec);

comp_n = n;
comp_d = d;
comp_o = o;
npml = convert(Int,λ/minimum(comp_d))*ones(Int64,(2,3));
npml = 10*ones(Int64,(2,3));
scheme = Waveform.helm3d_operto27;
cut_pml = true;
implicit_matrix = true;
srcfreqmask = trues(nsrc,nfreq);
misfit = Waveform.least_squares;
lsopts = LinSolveOpts(solver=:fgmres,maxit=1,maxinnerit=5,outputfreq=1);
lsopts.precond = :mlgmres;
opts = PDEopts{Int64,Float64}(scheme,comp_n,comp_d,comp_o,cut_pml,implicit_matrix,npml,misfit,srcfreqmask,lsopts);

nλ = maximum(n.*d/λ);
ppλ = λ/maximum(d);
@printf("Number of λ %3.3f , points per λ %3.3f",nλ,ppλ)
v = v0*ones(Float64,n...);
v = vec(v);
opts.implicit_matrix = true;

(H,comp_grid,T,DT_adj,P) = helmholtz_system(v,model,freq,opts);


In [ ]:
using MATLAB
z = H*y;
@matlab disp(1)
@mput model
@mput v
@mput y
@mput z
@mput t
@mput Pt
eval_string("
    lsopts = LinSolveOpts();
    lsopts.tol = 1e-6;
    lsopts.maxit = 1;
    lsopts.maxinnerit = 5;
    lsopts.solver = LinSolveOpts.SOLVE_FGMRES;
    lsopts.precond = LinSolveOpts.PREC_MLGMRES;
    model.n = vec(model.n)';
    model.d = vec(model.d)';
    model.o = vec(model.o)';
    v = 2000*ones(model.n);
    opts = struct;
    opts.dt = model.d;
    opts.pml = 10;
    opts.mat_free = true;
    opts.n_threads = 1;
    opts.scheme = PDEopts.HELM3D_OPERTO27;
    opts.solve_opts = lsopts;
")
eval_string("[H,comp_grid] = discrete_helmholtz(v,model,model.freq,opts);")
eval_string("tic,Pt1=H.solve_opts.precond*t;disp(toc)")
eval_string("norm(Pt1-Pt)")